
# Black–Litterman + Multi‑Benchmark Tracking Error (MBTE) Optimisation

**Optimizers:** Mean‑Variance, Max‑Sharpe, Higher Moments (skewness & kurtosis)  
**Data:** Downloaded via `yfinance` (daily, Adjusted Close)


### Returns & Covariance
Let asset returns be $r_t \in \mathbb{R}^n$ with sample mean $\hat\mu$ and covariance $\hat\Sigma$. We use log returns by default:

$$
r_{t} = \ln\frac{P_{t}}{P_{t-1}}, \quad 
\hat\mu = \frac{1}{T}\sum_{t=1}^{T} r_t, \quad 
\hat\Sigma = \frac{1}{T-1}\sum_{t=1}^{T}(r_t-\hat\mu)(r_t-\hat\mu)^\top.
$$

### Black–Litterman
Let $w^{\mathrm{mkt}}$ denote a market portfolio (e.g., cap‑weighted proxy). Reverse‑optimized (equilibrium) returns:

$$
\pi = \delta\,\hat\Sigma\, w^{\mathrm{mkt}},
$$

with risk aversion $\delta > 0$. Investor views are

$$
P \in \mathbb{R}^{k\times n},\quad Q \in \mathbb{R}^{k},\quad \Omega \in \mathbb{R}^{k\times k}.
$$

The Black–Litterman posterior mean (Theil mixed estimator):

$$
\mu_{\mathrm{BL}} = \Big[(\tau \hat\Sigma)^{-1} + P^\top \Omega^{-1} P\Big]^{-1}
\Big[(\tau \hat\Sigma)^{-1}\pi + P^\top \Omega^{-1} Q\Big],
$$

where $\tau \in (0,1]$ scales prior uncertainty. We use $\Sigma_{\mathrm{BL}} = \hat\Sigma$ by default.

### Multi‑Benchmark Tracking Error (MBTE)
Let benchmark returns be $r^{(b)}_t \in \mathbb{R}^m$ (e.g., SPY, AGG, VEA). With a convex combo of benchmarks $\alpha \in \mathbb{R}^m$, $\alpha \ge 0, \ \mathbf{1}^\top\alpha=1$, the portfolio return $r^p_t = w^\top r_{t}$ and composite benchmark $r^{B}_t = \alpha^\top r^{(b)}_t$.

**Tracking error variance (TEV):**

$$
\mathrm{TEV}(w,\alpha) = \mathbb{V}\mathrm{ar}(r^p_t - r^{B}_t) = \begin{bmatrix} w \\ -\alpha \end{bmatrix}^\top \Sigma_{\mathrm{all}} \begin{bmatrix} w \\ -\alpha \end{bmatrix},
$$

where $\Sigma_{\mathrm{all}}$ is the full covariance across $[r_t, r^{(b)}_t]$.

We include MBTE as a penalty term $\tfrac{\eta}{2}\,\mathrm{TEV}(w,\alpha)$ in objectives.

### Optimisation Problems
1. **Mean‑Variance (MV)** with BL means:
   $$
   \max_{w} \ \mu_{\mathrm{BL}}^\top w - \frac{\gamma}{2}\, w^\top \hat\Sigma\, w - \frac{\eta}{2}\,\mathrm{TEV}(w,\alpha)
   \quad\text{s.t.}\quad \mathbf{1}^\top w = 1,\ \ell \le w \le u.
   $$
2. **Max‑Sharpe (MSR)** with BL means:
   $$
   \max_{w} \ \frac{\mu_{\mathrm{BL}}^\top w - r_f}{\sqrt{w^\top \hat\Sigma\, w}} - \frac{\eta}{2}\,\mathrm{TEV}(w,\alpha).
   $$
3. **Higher‑Moments Utility (HM)** using sample skewness and excess kurtosis:
   $$
   \max_{w}\ U(w) = \mu_{\mathrm{BL}}^\top w - \frac{\lambda_2}{2}\, \sigma^2(w) + \lambda_3\, \mathrm{Skew}(w) - \lambda_4\, \mathrm{ExKurt}(w) - \frac{\eta}{2}\,\mathrm{TEV}(w,\alpha).
   $$

---

> **Edit the config cell below to use your own tickers, views, bounds, and penalty weights.**


In [ ]:
!pip install cvxpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--Downloading cvxpy-1.7.2-cp313-cp313-macosx_10_13_universal2.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/935.1 kB ? eta -:--:--Downloading clarabel-0.11.1-cp39-abi3-macosx_11_0_arm64.whl (935 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.1/935.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [cvxpy]32m3/4 [cvxpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [cvxpy]


In [2]:

# !pip install yfinance cvxpy scipy numpy pandas matplotlib

import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from scipy.optimize import minimize
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: f'{x:,.6f}')


In [3]:

# =================== CONFIG ===================
ASSETS = ['AAPL','MSFT','GOOGL','AMZN','NVDA','JPM','XOM','UNH']
BENCHMARKS = ['SPY','AGG','VEA']

START = '2018-01-01'
END   = None

RISK_FREE_ANNUAL = 0.02
TRADING_DAYS = 252

gamma = 5.0   # MV risk aversion
eta   = 5.0   # MBTE penalty

lambda2 = 5.0 # variance penalty
lambda3 = 1.0 # skew reward
lambda4 = 1.0 # excess kurt penalty

long_only = True
w_lower = 0.0 if long_only else -1.0
w_upper = 1.0

delta = 3.0
tau   = 0.05

w_mkt_default = None  # set list/array to override

P = np.array([
    [1, 1, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0,-1, 0],
], dtype=float)
Q = np.array([0.03, -0.01])
Omega = np.diag([0.02**2, 0.015**2])

alpha = np.array([0.6, 0.2, 0.2])
assert abs(alpha.sum()-1.0) < 1e-9, "alpha must sum to 1"
# ==============================================


In [4]:

tickers_all = ASSETS + BENCHMARKS
px = yf.download(tickers_all, start=START, end=END, auto_adjust=True, progress=False)['Close']
px = px.dropna()

ret = np.log(px/px.shift(1)).dropna()

R_assets = ret[ASSETS].copy()
R_bench  = ret[BENCHMARKS].copy()

mu_assets_ann = R_assets.mean()*TRADING_DAYS
Sigma_assets_ann = R_assets.cov()*TRADING_DAYS

R_all = pd.concat([R_assets, R_bench], axis=1)
Sigma_all_ann = R_all.cov()*TRADING_DAYS

n = len(ASSETS); m = len(BENCHMARKS)
print("Data window:", R_assets.index.min().date(), "to", R_assets.index.max().date())
print("Assets:", ASSETS)
print("Benchmarks:", BENCHMARKS)
Sigma_assets_ann.head()


Data window: 2018-01-03 to 2025-09-19
Assets: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'JPM', 'XOM', 'UNH']
Benchmarks: ['SPY', 'AGG', 'VEA']


Ticker,AAPL,MSFT,GOOGL,AMZN,NVDA,JPM,XOM,UNH
Ticker,,,,,,,,
AAPL,0.096805,0.063223,0.061088,0.064852,0.093146,0.039879,0.030141,0.032623
MSFT,0.063223,0.082010,0.064548,0.068357,0.098763,0.037892,0.024011,0.033676
GOOGL,0.061088,0.064548,0.096549,0.070052,0.094111,0.038742,0.027104,0.030764
AMZN,0.064852,0.068357,0.070052,0.118989,0.105558,0.033370,0.019740,0.024208
NVDA,0.093146,0.098763,0.094111,0.105558,0.267412,0.055441,0.034754,0.039315


In [5]:

def black_litterman(mu_sample, Sigma, w_mkt=None, delta=3.0, tau=0.05, P=None, Q=None, Omega=None):
    # Classic BL uses equilibrium returns pi = delta * Sigma * w_mkt
    n = Sigma.shape[0]
    if w_mkt is None:
        w_mkt = np.ones(n)/n
    w_mkt = np.asarray(w_mkt).reshape(-1)

    pi = delta * (Sigma @ w_mkt)

    if P is None or Q is None:
        return pi

    P = np.asarray(P, dtype=float)
    Q = np.asarray(Q, dtype=float).reshape(-1)
    if Omega is None:
        Omega = np.diag(np.diag(P @ (tau*Sigma) @ P.T))

    inv_tauSigma = np.linalg.inv(tau*Sigma)
    mid = inv_tauSigma + P.T @ np.linalg.inv(Omega) @ P
    rhs = inv_tauSigma @ pi + P.T @ np.linalg.inv(Omega) @ Q
    mu_bl = np.linalg.solve(mid, rhs)
    return mu_bl

w_mkt = w_mkt_default if w_mkt_default is not None else np.ones(n)/n
mu_BL = black_litterman(mu_assets_ann.values, Sigma_assets_ann.values, w_mkt=w_mkt,
                        delta=delta, tau=tau, P=P, Q=Q, Omega=Omega)
mu_BL = pd.Series(mu_BL, index=ASSETS, name="mu_BL (annual)")
mu_BL


AAPL    0.009308
MSFT    0.011652
GOOGL   0.010216
AMZN    0.025829
NVDA    0.002814
JPM     0.030652
XOM     0.012182
UNH     0.040454
Name: mu_BL (annual), dtype: float64

In [6]:

def tev_quadratic(w, alpha, Sigma_all, n, m):
    w = np.asarray(w).reshape(-1)
    alpha = np.asarray(alpha).reshape(-1)
    z = np.concatenate([w, -alpha])
    return float(z.T @ Sigma_all.values @ z)

def portfolio_stats(w, mu_series, Sigma):
    w = np.asarray(w).reshape(-1)
    mu = float(mu_series.values @ w)
    var = float(w.T @ Sigma.values @ w)
    vol = np.sqrt(var)
    return mu, vol, var

def sample_portfolio_moments(w, R_assets, annualize=True, trading_days=252):
    w = np.asarray(w).reshape(-1)
    rp = R_assets.values @ w
    if annualize:
        mu = rp.mean()*trading_days
        vol = rp.std(ddof=1)*np.sqrt(trading_days)
        from scipy.stats import skew, kurtosis
        sk = skew(rp, bias=False)
        exk = kurtosis(rp, fisher=True, bias=False)
        return mu, vol, sk, exk, pd.Series(rp, index=R_assets.index, name="rp")
    else:
        from scipy.stats import skew, kurtosis
        mu = rp.mean()
        vol = rp.std(ddof=1)
        sk = skew(rp, bias=False)
        exk = kurtosis(rp, fisher=True, bias=False)
        return mu, vol, sk, exk, pd.Series(rp, index=R_assets.index, name="rp")


In [17]:
!pip install ecos
import ecos

In [ ]:
# Ensure ECOS is installed for cvxpy
try:
    import ecos
except ImportError:
    import sys
    !{sys.executable} -m pip install ecos

import cvxpy as cp

def optimise_mv_mbte(mu, Sigma, Sigma_all, alpha, gamma=5.0, eta=1.0, lower=0.0, upper=1.0):
    n = len(mu)
    w = cp.Variable(n)
    ones = np.ones(n)

    Q = Sigma_all.values
    Q_aa = Q[:n,:n]
    Q_ab = Q[:n,n:]

    var_term = cp.quad_form(w, Sigma.values)
    tev_quad = cp.quad_form(w, Q_aa) - 2* (Q_ab @ alpha) @ w

    obj = cp.Maximize(mu.values @ w - 0.5*gamma*var_term - 0.5*eta*tev_quad)
    cons = [cp.sum(w) == 1]
    if lower is not None: cons.append(w >= lower)
    if upper is not None: cons.append(w <= upper)

    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.ECOS, verbose=False)
    return np.array(w.value).reshape(-1), prob.value, prob.status

w_mv, obj_mv, status_mv = optimise_mv_mbte(mu_BL, Sigma_assets_ann, Sigma_all_ann, alpha,
                                           gamma=gamma, eta=eta, lower=w_lower, upper=w_upper)
mu_mv, vol_mv, _ = portfolio_stats(w_mv, mu_BL, Sigma_assets_ann)
sharpe_mv = (mu_mv - RISK_FREE_ANNUAL)/vol_mv
w_mv_series = pd.Series(w_mv, index=ASSETS, name="MV+MBTE weights")
status_mv, obj_mv, mu_mv, vol_mv, sharpe_mv, w_mv_series


SolverError: The solver ECOS is not installed.

In [15]:
# Ensure ECOS is installed for cvxpy
try:
    import ecos
except ImportError:
    import sys
    !{sys.executable} -m pip install ecos

import cvxpy as cp

def optimise_max_sharpe_mbte(mu, Sigma, Sigma_all, alpha, rf=0.0, eta=1.0, lower=0.0, upper=1.0):
    n = len(mu)
    w = cp.Variable(n)
    t = cp.Variable()

    Q = Sigma_all.values
    Q_aa = Q[:n,:n]
    Q_ab = Q[:n,n:]
    tev_quad = cp.quad_form(w, Q_aa) - 2* (Q_ab @ alpha) @ w

    c = 1.0
    S = np.linalg.cholesky(Sigma.values + 1e-10*np.eye(n))
    obj = cp.Minimize(-(mu.values @ w - rf) + c*t + 0.5*eta*tev_quad)
    cons = [
        cp.sum(w) == 1,
        cp.SOC(t, S @ w)
    ]
    if lower is not None: cons.append(w >= lower)
    if upper is not None: cons.append(w <= upper)

    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.ECOS, verbose=False)
    return np.array(w.value).reshape(-1), prob.value, prob.status

w_msr, obj_msr, status_msr = optimise_max_sharpe_mbte(mu_BL, Sigma_assets_ann, Sigma_all_ann, alpha,
                                                       rf=RISK_FREE_ANNUAL, eta=eta, lower=w_lower, upper=w_upper)
mu_msr, vol_msr, _ = portfolio_stats(w_msr, mu_BL, Sigma_assets_ann)
sharpe_msr = (mu_msr - RISK_FREE_ANNUAL)/vol_msr
w_msr_series = pd.Series(w_msr, index=ASSETS, name="MaxSharpe+MBTE weights")
status_msr, obj_msr, mu_msr, vol_msr, sharpe_msr, w_msr_series


SolverError: The solver ECOS is not installed.

In [ ]:

def hm_utility(w, mu_series, Sigma, R_assets, Sigma_all, alpha,
               lambda2=5.0, lambda3=1.0, lambda4=1.0, eta=1.0):
    mu, vol, sk, exk, rp = sample_portfolio_moments(w, R_assets, annualize=True)
    tev = tev_quadratic(w, alpha, Sigma_all, len(mu_series), len(alpha))
    U = (mu_series.values @ w) - 0.5*lambda2*(vol**2) + lambda3*sk - lambda4*exk - 0.5*eta*tev
    return -float(U)

def optimise_higher_moments(mu_series, Sigma, R_assets, Sigma_all, alpha,
                            lambda2=5.0, lambda3=1.0, lambda4=1.0, eta=1.0,
                            lower=0.0, upper=1.0):
    n = len(mu_series)
    x0 = np.ones(n)/n
    bounds = [(lower, upper)]*n
    cons = ({'type': 'eq','fun': lambda w: np.sum(w) - 1.0},)

    res = minimize(hm_utility, x0, method='SLSQP', bounds=bounds, constraints=cons,
                   args=(mu_series, Sigma, R_assets, Sigma_all, alpha,
                         lambda2, lambda3, lambda4, eta),
                   options={'maxiter': 1000, 'ftol': 1e-9})
    return res

res_hm = optimise_higher_moments(mu_BL, Sigma_assets_ann, R_assets, Sigma_all_ann, alpha,
                                 lambda2=lambda2, lambda3=lambda3, lambda4=lambda4, eta=eta,
                                 lower=w_lower, upper=w_upper)
w_hm = res_hm.x
mu_hm, vol_hm, _ = portfolio_stats(w_hm, mu_BL, Sigma_assets_ann)
from scipy.stats import skew, kurtosis
_, _, sk_hm, exk_hm, rp_hm = sample_portfolio_moments(w_hm, R_assets, annualize=True)
sharpe_hm = (mu_hm - RISK_FREE_ANNUAL)/vol_hm
w_hm_series = pd.Series(w_hm, index=ASSETS, name="HigherMoments+MBTE weights")
res_hm.success, res_hm.fun, mu_hm, vol_hm, sharpe_hm, sk_hm, exk_hm, w_hm_series


In [ ]:

summary = pd.DataFrame({
    'MV+MBTE': [mu_mv, vol_mv, sharpe_mv],
    'MaxSharpe+MBTE': [mu_msr, vol_msr, sharpe_msr],
    'HigherMoments+MBTE': [mu_hm, vol_hm, sharpe_hm],
}, index=['Ann. Return (BL μ)', 'Ann. Vol', 'Sharpe (rf=2%)'])

weights = pd.concat([w_mv_series, w_msr_series, w_hm_series], axis=1)
summary, weights.round(4)


In [ ]:

plt.figure()
plt.scatter(vol_mv, mu_mv, label='MV+MBTE')
plt.scatter(vol_msr, mu_msr, label='MaxSharpe+MBTE')
plt.scatter(vol_hm, mu_hm, label='HigherMoments+MBTE')
plt.xlabel('Annual Volatility')
plt.ylabel('Annual Return (BL μ)')
plt.title('Optimised Portfolios')
plt.legend()
plt.show()

tev_mv  = tev_quadratic(w_mv,  alpha, Sigma_all_ann, len(ASSETS), len(BENCHMARKS))
tev_msr = tev_quadratic(w_msr, alpha, Sigma_all_ann, len(ASSETS), len(BENCHMARKS))
tev_hm  = tev_quadratic(w_hm,  alpha, Sigma_all_ann, len(ASSETS), len(BENCHMARKS))

pd.Series({'MV+MBTE': tev_mv, 'MaxSharpe+MBTE': tev_msr, 'HigherMoments+MBTE': tev_hm}, name='TEV').to_frame()


In [ ]:

def cum_return(series):
    return (1 + series).cumprod()

def portfolio_series(w, R_assets):
    rp = (R_assets.values @ w)
    return pd.Series(rp, index=R_assets.index, name='rp')

rp_mv  = portfolio_series(w_mv, R_assets)
rp_msr = portfolio_series(w_msr, R_assets)
rp_hm  = portfolio_series(w_hm, R_assets)
rb     = (R_bench.values @ alpha)
rb = pd.Series(rb, index=R_bench.index, name='rb')

cr_mv  = cum_return(rp_mv)
cr_msr = cum_return(rp_msr)
cr_hm  = cum_return(rp_hm)
cr_b   = cum_return(rb)

plt.figure()
plt.plot(cr_mv, label='MV+MBTE')
plt.plot(cr_msr, label='MaxSharpe+MBTE')
plt.plot(cr_hm, label='HigherMoments+MBTE')
plt.plot(cr_b, label='Composite Benchmark (α)', linestyle='--')
plt.title('Cumulative Return (static weights)')
plt.legend()
plt.show()
